<a href="https://colab.research.google.com/github/debbysonino/LamasDataHack/blob/master/SubVariables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import section

import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import missingno

In [0]:
# General DS libraries we are going to need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Importing our base model
# [REDACTED ML MODEL USED]

# Imports for working with our large dataset
from sklearn.utils.random import sample_without_replacement
from sklearn.model_selection import train_test_split

# We need those for data manipulation and getting our features ready for the model
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer

# These can be used to measure our model's performance
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# Ignore DataFrame assignment warnings
pd.options.mode.chained_assignment = None

Data exploration

In [42]:
accounts_df = pd.read_csv('https://storage.googleapis.com/mondaycom-datahack/final_sets/train_accounts.csv', nrows=7000000000)
sub_df = pd.read_csv('https://storage.googleapis.com/mondaycom-datahack/final_sets/train_subscriptions.csv', nrows=7000000000)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
sub_df.head()


In [0]:
#sub_df['payment_type'].fillna('').value_counts()
sub_df['event_type'].fillna('').value_counts()



In [0]:
sub_df.account_id.isna().value_counts()

In [0]:
sub_df.shape

In [0]:
sub_df.account_id.value_counts()

In [55]:
sub_df.columns

Index(['event_happened_at', 'subscription_id', 'account_id', 'plan_id',
       'event_type', 'invoice_charge_amount', 'prev_plan_id', 'status',
       'status_reason', 'currency', 'invoice_charge_amount_usd', 'mrr_gain',
       'next_charge_date', 'payment_type', 'transaction_date'],
      dtype='object')

In [80]:
accounts_df.columns

Index(['account_id', 'account_name', 'created_at', 'plan_id', 'trial_start',
       'churn_date', 'churn_reason', 'time_diff', 'region', 'country',
       'subscription_started_at', 'paying', 'has_logo', 'device', 'os',
       'browser', 'collection_21_days', 'company_size', 'payment_currency',
       'max_team_size', 'min_team_size', 'industry', 'utm_cluster_id', 'mrr',
       'user_goal', 'user_description', 'team_size', 'lead_score'],
      dtype='object')

## sub variables of accounts

In [0]:
#sub_df = pd.read_csv('https://storage.googleapis.com/mondaycom-datahack/final_sets/train_subscriptions.csv', nrows=7000000000)
def upgrade(row):
  return abs(row["plan_id"]-row["prev_plan_id"])>0


sub_df["plan_upgrade"] =  sub_df.apply(upgrade ,axis=1)

sub_df['status_success']=sub_df['status'].fillna('SUCCESS')
sub_df['status_success']=sub_df['status_success']=='SUCCESS'
#sub_df[sub_df['status_success']==0].sum() # check if mrr=0 for status=failed

account_subs=sub_df.groupby('account_id').agg({"subscription_id":"count","plan_id":"nunique", "plan_upgrade": "mean","mrr_gain":"sum"})

In [79]:
account_subs.head()

,subscription_id,plan_id,plan_upgrade,mrr_gain
account_id,,,,
13.0,3,1,0.000000,0.0
72.0,7,3,0.285714,49.0
209.0,1,1,0.000000,39.0
235.0,3,2,0.333333,49.0
294.0,1,1,0.000000,39.0


In [0]:
agg_df = sub_df.groupby('account_id').agg({"plan_id":"nunique", "plan_upgrade": "mean","status_success":"mean"})
agg_df

In [0]:
positives = accounts_df[accounts_df["lead_score"] == 1]

negatives = accounts_df[accounts_df["lead_score"] == 0].sample(positives.shape[0])

In [0]:
accounts_df_sample = pd.concat([positives, negatives])

In [19]:
#((~accounts_df_sample["mrr"].isna()) == (accounts_df_sample["paying"])).mean()

0.9704329617176901

In [0]:
accounts_df_sample.head(20)


## Data exploration

In [0]:
plt.figure(figsize=(15, 10))
sns.heatmap(accounts_df_sample.corr(), annot=True)

In [0]:
less_missing = accounts_df_sample[accounts_df_sample.columns[accounts_df_sample.isna().sum() < accounts_df_sample.shape[0]*0.8]]

In [25]:
less_missing.shape
accounts_df_sample.shape


(67812, 28)

In [0]:
accounts_df_sample.nunique()

In [0]:
index_of_useless_columns = less_missing.nunique().isin([0,1])
columns_to_drop = np.array(less_missing.columns)[index_of_useless_columns]
less_missing.drop(columns=columns_to_drop, inplace=True)

In [33]:
less_missing.shape

(67812, 22)

In [0]:
time_between_created_trail = pd.to_datetime(less_missing['trial_start']) - pd.to_datetime(less_missing['created_at'])

In [0]:
less_missing = less_missing.assign(created_trial_delta=time_between_created_trail.apply(lambda x: (x.seconds//3600)))

In [41]:
less_missing.head(30)


,account_id,account_name,created_at,trial_start,time_diff,region,country,paying,device,os,browser,collection_21_days,company_size,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,user_goal,user_description,team_size,lead_score,created_trial_delta
131,140.0,Jones-Macdonald,2019-01-01 01:40:36,2019-01-01 01:40:36,NaN,Florida,US,0,mobile,ios,NaN,0,1130.0,USD,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1,0
193,205.0,"Clark, Rocha and Carter",2019-01-01 02:37:08,2019-01-01 02:37:08,NaN,Illinois,US,0,mobile,ios,NaN,0,990.0,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,1,0
249,266.0,Taylor Group,2019-01-01 03:41:19,2019-01-01 03:41:19,NaN,Pennsylvania,US,0,mobile,ios,safari,0,2640.0,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
254,272.0,Garrett-Kline,2019-01-01 03:47:09,2019-01-01 03:47:09,10.0,Queensland,AU,0,desktop,windows,chrome,0,1200.0,AUD,1.0,1.0,IT,todos,NaN,NaN,1-1,1,0
313,336.0,Gonzales-Acevedo,2019-01-01 05:14:21,2019-01-01 05:14:21,-7.0,Colorado,US,0,mobile,ios,generic browser,0,350.0,USD,1.0,1.0,NaN,time_management,NaN,NaN,NaN,1,0
324,348.0,"Martin, Wood and Lin",2019-01-01 05:28:30,2019-01-01 05:28:30,-5.0,New Jersey,US,0,desktop,mac,chrome,0,80730.0,USD,100.0,51.0,BI,product_roadmap,NaN,NaN,51-100,1,0
354,379.0,Davidson Ltd,2019-01-01 06:08:51,2019-01-01 06:08:51,NaN,Gyeonggi-do,KR,0,mobile,ios,NaN,0,89300.0,USD,50.0,26.0,NaN,NaN,NaN,NaN,NaN,1,0
357,382.0,"Jacobs, Gutierrez and Brown",2019-01-01 06:11:01,2019-01-01 06:11:01,11.0,Victoria,AU,0,mobile,android,chrome,0,360.0,AUD,500.0,101.0,NaN,NaN,NaN,NaN,NaN,1,0
384,411.0,Roberts-Luna,2019-01-01 06:41:08,2019-01-01 06:41:08,-7.0,Utah,US,1,desktop,mac,chrome,1428,1900.0,USD,10.0,6.0,Project Management,internal_projects,NaN,NaN,6-10,1,0
398,426.0,"Hendrix, Jones and Baker",2019-01-01 07:00:38,2019-01-01 07:00:38,NaN,Karnataka,IN,0,mobile,ios,NaN,0,25000.0,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,1,0
